<a href="https://colab.research.google.com/github/Neilus03/Task-assignment-problem/blob/main/XGBOOST_synthesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import xgboost as xgb


In [ ]:
# Cargar el conjunto de datos
data = pd.read_excel('/content/drive/MyDrive/tasks_data.xlsx')

In [ ]:
data.head()

,PROJECT_ID,PM,TASK_ID,START,END,TASK_TYPE,SOURCE_LANG,TARGET_LANG,TRANSLATOR,ASSIGNED,...,CLOSE,FORECAST,HOURLY_RATE,COST,QUALITY_EVALUATION,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY
0,213494,KMT,10048285,2014-11-26 10:36:00,2014-11-26 15:30:00,Engineering,English,Portuguese (Brazil),Estela,2014-11-26 16:34:23,...,2014-11-26 17:51:48,0.25,24,6.00,7,Coastal Cottage,Consumer Discretionary,Consumer Services,"Hotels, Restaurants & Leisure","Hotels, Resorts & Cruise Lines"
1,214198,KMT,10048285,2015-09-09 17:29:00,2015-09-10 11:00:00,Engineering,English,Spanish (Iberian),Jeronimo,2015-09-09 17:31:52,...,2015-09-10 17:33:07,1.50,20,30.00,7,HealthyLife,Health Care,Health Care Providers,Health Care Facilities,Long-Term Care Facilities
2,213094,PMT,10048285,2014-05-01 19:27:00,2014-05-02 19:00:00,Engineering,Catalan,Catalan,Octavi,2014-05-02 13:29:33,...,2014-05-02 13:29:44,0.33,15,4.95,6,AeroSysTech,Information Technology,Software & Services,Application Software,Systems Software
3,211967,KMT,10048285,2013-07-17 17:40:00,2013-08-31 18:00:00,Management,English,Spanish (Iberian),Ramiro Josafat,2013-07-22 15:35:11,...,2013-08-31 16:28:18,0.50,20,10.00,6,MetaPro,Industrials,Electrical Equipment,Electrical Components & Equipment,Electrical Components & Equipment
4,212331,PMT,10048285,2013-11-01 13:13:00,2013-10-31 19:00:00,Miscellaneous,Catalan,Catalan,Victor,2013-10-31 13:14:35,...,2013-10-31 13:18:34,0.00,11,0.00,8,SunTech,Information Technology,Technology Hardware & Equipment,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals"


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 997947 entries, 0 to 997946
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   PROJECT_ID                   997947 non-null  object        
 1   PM                           997947 non-null  object        
 2   TASK_ID                      997947 non-null  int64         
 3   START                        997947 non-null  object        
 4   END                          997947 non-null  object        
 5   TASK_TYPE                    997947 non-null  object        
 6   SOURCE_LANG                  997947 non-null  object        
 7   TARGET_LANG                  997947 non-null  object        
 8   TRANSLATOR                   997947 non-null  object        
 9   ASSIGNED                     997947 non-null  datetime64[ns]
 10  READY                        997947 non-null  datetime64[ns]
 11  WORKING                   

In [ ]:
working_data = data.sample(300000)

In [ ]:
working_data = working_data.drop(columns=[ 'ASSIGNED', 'READY', 'WORKING', 'DELIVERED', 'RECEIVED', 'CLOSE', 'PROJECT_ID', 'TASK_ID', 'START', 'END'])

In [ ]:
working_data.head()

,PM,TASK_TYPE,SOURCE_LANG,TARGET_LANG,TRANSLATOR,FORECAST,HOURLY_RATE,COST,QUALITY_EVALUATION,MANUFACTURER,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY
697947,KMT,Miscellaneous,English,Spanish (Iberian),Ascension,0.80,14,11.20,7,Infusion Dynamics,Health Care,Health Care Equipment & Supplies,Health Care Equipment & Supplies,Health Care Equipment & Supplies
697948,PMT,ProofReading,German,Spanish (Iberian),Luis Felipe,0.09,20,1.80,8,Penta Dynamics,Industrials,Machinery,Industrial Machinery,Industrial Machinery
697949,BMT,PostEditing,English,Catalan,Oscar,2.25,20,45.00,7,AeroSysTech,Information Technology,Software & Services,Application Software,Systems Software
697950,KMT,ProofReading,English,Spanish (Iberian),Ascension,3.32,14,46.48,7,BioFabrica,Health Care,Health Care Equipment & Services,Health Care Equipment,Life Sciences Tools & Services
697951,BMT,Translation,English,Basque,Celso,3.46,25,86.50,5,AeroSysTech,Information Technology,Software & Services,Application Software,Systems Software


In [ ]:
working_data = working_data.drop(columns=['PM', 'MANUFACTURER'])

In [ ]:
working_data.head()

,TASK_TYPE,SOURCE_LANG,TARGET_LANG,TRANSLATOR,FORECAST,HOURLY_RATE,COST,QUALITY_EVALUATION,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY
697947,Miscellaneous,English,Spanish (Iberian),Ascension,0.80,14,11.20,7,Health Care,Health Care Equipment & Supplies,Health Care Equipment & Supplies,Health Care Equipment & Supplies
697948,ProofReading,German,Spanish (Iberian),Luis Felipe,0.09,20,1.80,8,Industrials,Machinery,Industrial Machinery,Industrial Machinery
697949,PostEditing,English,Catalan,Oscar,2.25,20,45.00,7,Information Technology,Software & Services,Application Software,Systems Software
697950,ProofReading,English,Spanish (Iberian),Ascension,3.32,14,46.48,7,Health Care,Health Care Equipment & Services,Health Care Equipment,Life Sciences Tools & Services
697951,Translation,English,Basque,Celso,3.46,25,86.50,5,Information Technology,Software & Services,Application Software,Systems Software


In [ ]:
# Codificación one-hot para variables categóricas
categorical_columns = ['TASK_TYPE', 'SOURCE_LANG', 'TARGET_LANG',
                       'MANUFACTURER_SECTOR', 'MANUFACTURER_INDUSTRY_GROUP',
                       'MANUFACTURER_INDUSTRY', 'MANUFACTURER_SUBINDUSTRY']

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(working_data[categorical_columns])

# Normalizar variables numéricas
numeric_columns = ['HOURLY_RATE', 'COST', 'FORECAST']
scaler = StandardScaler()
scaled_numeric = scaler.fit_transform(working_data[numeric_columns])

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
working_data.head()

,TASK_TYPE,SOURCE_LANG,TARGET_LANG,TRANSLATOR,FORECAST,HOURLY_RATE,COST,QUALITY_EVALUATION,MANUFACTURER_SECTOR,MANUFACTURER_INDUSTRY_GROUP,MANUFACTURER_INDUSTRY,MANUFACTURER_SUBINDUSTRY
697947,Miscellaneous,English,Spanish (Iberian),Ascension,0.80,14,11.20,7,Health Care,Health Care Equipment & Supplies,Health Care Equipment & Supplies,Health Care Equipment & Supplies
697948,ProofReading,German,Spanish (Iberian),Luis Felipe,0.09,20,1.80,8,Industrials,Machinery,Industrial Machinery,Industrial Machinery
697949,PostEditing,English,Catalan,Oscar,2.25,20,45.00,7,Information Technology,Software & Services,Application Software,Systems Software
697950,ProofReading,English,Spanish (Iberian),Ascension,3.32,14,46.48,7,Health Care,Health Care Equipment & Services,Health Care Equipment,Life Sciences Tools & Services
697951,Translation,English,Basque,Celso,3.46,25,86.50,5,Information Technology,Software & Services,Application Software,Systems Software


In [ ]:
# Combinar variables codificadas y escaladas
X = np.hstack([onehot_encoded, scaled_numeric])
y = working_data['QUALITY_EVALUATION'].values

In [ ]:

# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Entrenamiento del modelo XGBoost
model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

# Función para recomendar traductores
def recommend_translators_xgb(task, translator_list, model, onehot_encoder, scaler):
    task_data = []
    for translator in translator_list:
        # Crear un conjunto de datos con las características de la tarea y el traductor
        task_translator = task.copy()
        task_translator['TRANSLATOR'] = translator
        task_data.append(task_translator)

    task_df = pd.DataFrame(task_data)

    # Preprocesar los datos de la tarea utilizando el onehot_encoder y el scaler
    onehot_encoded = onehot_encoder.transform(task_df[categorical_columns])
    scaled_numeric = scaler.transform(task_df[numeric_columns])
    task_X = np.hstack([onehot_encoded, scaled_numeric])

    # Predecir la calidad de la traducción para cada traductor
    quality_predictions = model.predict(task_X)

    # Ordenar los traductores en función de la calidad de la traducción
    sorted_indices = np.argsort(-quality_predictions)
    top_translators = np.array(translator_list)[sorted_indices][:3]
    top_quality = quality_predictions[sorted_indices][:3]

    return list(zip(top_translators, top_quality))

In [ ]:
task = {
    'TASK_TYPE': 'Engineering',
    'SOURCE_LANG': 'Catalan',
    'TARGET_LANG': 'Portuguese (Brazil)',
    'MANUFACTURER_SECTOR': 'Consumer Discretionary',
    'MANUFACTURER_INDUSTRY_GROUP': 'Consumer Services',
    'MANUFACTURER_INDUSTRY': 'Hotels, Restaurants & Leisure',
    'MANUFACTURER_SUBINDUSTRY': 'Hotels, Resorts & Cruise Lines',
    'HOURLY_RATE': 20,
    'COST': 21,
    'FORECAST': 0.25
}

translator_list = working_data['TRANSLATOR'].unique()

recommendations = recommend_translators_xgb(task, translator_list, model, onehot_encoder, scaler)

print("Recomendaciones:")
for translator, quality in recommendations:
    print(f"Traductor: {translator}, Calidad estimada: {quality}")


Recomendaciones:
Traductor: Acacio Poncio, Calidad estimada: 7.0088419914245605
Traductor: Ivan, Calidad estimada: 7.0088419914245605
Traductor: Lionel Yamila, Calidad estimada: 7.0088419914245605
